# Processamento de dados sonometro

## Ler ficheiro

In [41]:
import struct

data_size = 4
time_size = 1

fn = "example.bin"

bin_file = open(fn,mode="rb")

header = bin_file.read(5) # Start header
if (header != b'start'): 
    raise(Exception("Wrong header: " + str(header)))
else:
    print(str(header))

bin_file.read(1) #Spacing byte

date = bin_file.read(6) # read start date from header

print("Start date: %d-%d-%d %dh%dm%ds\n" % (
    date[0]+2000,
    date[1],
    date[2],
    date[3],
    date[4],
    date[5])
)

data = bin_file.read()
s = ' '.join(map('{:02X}'.format, data))
print(s)

f_l = []

increment = data_size + time_size

while (data):
    s = ' '.join(map('{:02X}'.format, data[0:increment]))
    f = data[0:data_size]
    t = int(data[data_size:increment][0])
    data = data[increment:]
    db = struct.unpack('>f', f)[0]
    print(s)
    print("t: %d ms; v: %f dB \n" %(t,db))
    f_l.append(db)


'''
data = f.read(4) # float data
t = f.read(1) # timing data

while data:
    db = struct.unpack('>f', data)[0]

    print("ms: %d, val: %f db \n" % (int(t),db))

    data = f.read(4) # float data
    t = f.read(1) # timing data
'''

bin_file.close()

b'start'
Start date: 2024-11-12 18h34m16s

81 9A 8F 42 56 A8 1C 3A 42 EE C1 41 45 42 88 AC 2B 61 42 23 CC 23 3D 42 C0 6D 8F 3B 42 CA 70 5C 3C 42 5B D4 C8 3B 42 EC 4A 3C 4D 42 80 B2 A0 3B 42 14 0C 0C 41 42 AB AC 89 3B 42 42 18 D5 3B 42 DC 10 CC 3C 42 76 B1 DD 3C 42 12 C7 40 40 42 AF 9A 08 41 42 4F D0 A0 3D 42 EF 27 00 3F 42 92 38 DA 4F 42 B9 DD 4B 41 42 4D 0B 95 42 42 E5 B4 12 3E 42 7D
81 9A 8F 42 56
t: 86 ms; v: -0.000000 dB 

A8 1C 3A 42 EE
t: 238 ms; v: -0.000000 dB 

C1 41 45 42 88
t: 136 ms; v: -12.079409 dB 

AC 2B 61 42 23
t: 35 ms; v: -0.000000 dB 

CC 23 3D 42 C0
t: 192 ms; v: -42792200.000000 dB 

6D 8F 3B 42 CA
t: 202 ms; v: 5540999337999684668191932416.000000 dB 

70 5C 3C 42 5B
t: 91 ms; v: 272638197544242649066073227264.000000 dB 

D4 C8 3B 42 EC
t: 236 ms; v: -6879901122560.000000 dB 

4A 3C 4D 42 80
t: 128 ms; v: 3085136.500000 dB 

B2 A0 3B 42 14
t: 20 ms; v: -0.000000 dB 

0C 0C 41 42 AB
t: 171 ms; v: 0.000000 dB 

AC 89 3B 42 42
t: 66 ms; v: -0.000000 dB 

18 D5 3B 42

In [ ]:
# Import seaborn
import seaborn as sns

# Apply the default theme
sns.set_theme()

sns.lineplot(f_l)
